In [1]:
import pyodbc
from bs4 import BeautifulSoup
from urllib.parse import urljoin,urlparse
import requests
import re

In [2]:
conn=pyodbc.connect('Driver={SQL Server};''Server=DESKTOP-CDBHGSC;'
                    'Database=HousePriceMorocco;''Trusted_connection=yes')

In [3]:
if conn:
    print('True')

True


In [4]:
cursor = conn.cursor()

In [5]:
url = 'https://www.mubawab.ma/fr/cr/rabat-sal%C3%A9-zemmour-za%C3%ABr/immobilier-a-vendre-all' 
response = requests.get(url)
response

<Response [200]>

In [6]:
query = """
INSERT INTO Web (id_site, nom_site, url_site)
VALUES (?, ?, ?)
"""
cursor.execute(query,20, "mubawab", url)

In [7]:
soup = BeautifulSoup(response.content, 'html.parser')
div_elem = soup.find('div',class_="sideFilterWrap asideMarg")

In [8]:
ul_elems = div_elem.find('ul', class_="selectUl")
li_tags = ul_elems.find_all('li')
for li in li_tags:
    a_tag = li.find('a')
    link=a_tag.get('href')
    print(link)

https://www.mubawab.ma/fr/cr/b%C3%A9ni-mellal-kh%C3%A9nifra/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/casablanca-settat/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/dakhla-oued-eddahab/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/dar%C3%A2a-tafilalet/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/f%C3%A8s-mekn%C3%A8s/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/guelmim-oued-noun/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/l'oriental/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/la%C3%A2youne-sakia-el-hamra/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/marrakech-safi/immobilier-a-vendre-all
#
https://www.mubawab.ma/fr/cr/souss-massaa/immobilier-a-vendre-all
https://www.mubawab.ma/fr/cr/tanger-t%C3%A9touan-al-hoceima/immobilier-a-vendre-all


In [9]:
urls = [
    "https://www.mubawab.ma/fr/cr/b%C3%A9ni-mellal-kh%C3%A9nifra/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/casablanca-settat/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/dakhla-oued-eddahab/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/dar%C3%A2a-tafilalet/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/f%C3%A8s-mekn%C3%A8s/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/guelmim-oued-noun/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/l'oriental/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/la%C3%A2youne-sakia-el-hamra/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/marrakech-safi/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/souss-massaa/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/tanger-t%C3%A9touan-al-hoceima/immobilier-a-vendre-all"
]

In [10]:
num = 850
nb = 3200

urls = [
    "https://www.mubawab.ma/fr/cr/b%C3%A9ni-mellal-kh%C3%A9nifra/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/casablanca-settat/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/dakhla-oued-eddahab/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/dar%C3%A2a-tafilalet/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/f%C3%A8s-mekn%C3%A8s/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/guelmim-oued-noun/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/l'oriental/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/la%C3%A2youne-sakia-el-hamra/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/marrakech-safi/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/souss-massaa/immobilier-a-vendre-all",
    "https://www.mubawab.ma/fr/cr/tanger-t%C3%A9touan-al-hoceima/immobilier-a-vendre-all"
]

def extract_number(text):
    return ''.join(filter(str.isdigit, text))

for u in urls:
    response = requests.get(u)
    soup = BeautifulSoup(response.content, 'html.parser')
    division = soup.find_all('div', class_='col-7 contentBox')
    for dv in division:
        city = dv.find('a')
        if city:
            relative_link = city.get('href')
            base_url = u
            full_link = urljoin(base_url, relative_link)
            
            response = requests.get(full_link)
            soup = BeautifulSoup(response.content, 'html.parser')
            metrage = None
            piece = None
            price = None
            city_name = None
            quartier = None

            for feature in soup.find_all('div', class_='adDetailFeature'):
                span = feature.find('span')
                if span:
                    text = span.get_text(strip=True)
                    if 'm²' in text:
                        metrage = int(text.split()[0])
                    elif 'Pièce' in text:
                        piece = int(text.split()[0])
            

            price_tag = soup.find('h3', class_='fullPicturesPrice')
            if price_tag:
                text = price_tag.get_text(strip=True)
                if "Prix à consulter" in text:
                    price = 0
                else:
                    text = extract_number(text)
                    if text:
                        price = int(text)
            

            city_tag = soup.find('h3', class_='greyTit')
            if city_tag:
                city_text = city_tag.get_text(strip=True)
                if ' à ' in city_text:
                    quartier, city_name = map(str.strip, city_text.split(' à '))
                else:
                    parts = city_text.split('à')
                    if len(parts) == 2:
                        quartier = parts[0].strip()
                        city_name = parts[1].strip()
                    else:
                        city_name = city_text.strip()

            query = """
                    SELECT id_location
                    FROM Localisation 
                    WHERE city = ? AND (district = ? OR district IS NULL)
                    """
            cursor.execute(query, city_name, quartier)
            result = cursor.fetchone()
            
            if result:
                id_location = result[0]
            else:
                insert_location_query = """
                INSERT INTO Localisation (id_location,city, district) 
                VALUES (?,?, ?)
                """
                cursor.execute(insert_location_query,num, city_name, quartier)
                id_localisation = num
                num = num+1
            nb=nb+1
            insert_maison_query = """
            INSERT INTO House (id_house, price, number_of_rooms, area, id_site, id_location) 
            VALUES (?, ?, ?, ?, ?, ?)
            """
            cursor.execute(insert_maison_query, nb, price, piece, metrage, 20, id_localisation)

            

In [11]:
conn.commit()

In [12]:
cursor.close()
conn.close()